

### Based on the example from Tensorflow's text generation with RNN:
### https://www.tensorflow.org/text/tutorials/text_generation

### Text used for the RNN model: The entire script from Lord of the Rings: Fellowship of the Ring

# Changes made to the code:
## If there are modifications made for this code, then it would be the length of the example sequence for input text processing and the number of epochs to run for training. Also, the changes made so far are the example text vector ['abcdefghij', 'wxyz'] and the temperature parameter for the one-step RNN model class. For the latter, I choose the following values to try out: 1.0, 0.9, 0.8, 0.7, 0.6, and 0.5.

In [ ]:
# Import the following modules over
import numpy as np
import tensorflow as tf
import os
import time

# Try to open the text file if it exists.
input_file = "rings.txt"
corpus_raw = open(input_file, "r", encoding="utf-8").read()

# Print out the sample text file.
#print(corpus_raw)

In [ ]:
# Obtain the list of characters included in the raw text.
# Be sure to use set() to filter out duplicates.
characters = sorted(list(set(corpus_raw)))

# Print out the list of characters found in the raw text.
print(characters)

# Also obtain the total length of the text and the characters.
print("Total length of text: ", len(corpus_raw))
print("Total number of characters found: ", len(characters))

['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Ó', 'á', 'â', 'ä', 'é', 'ë', 'í', 'ó', 'ú', 'û', '–']
Total length of text:  1021058
Total number of characters found:  90


In [ ]:
# Print out the first 500 characters of the text.
print(corpus_raw[:500])

Three Rings for the Elven-kings under the sky,
               Seven for the Dwarf-lords in their halls of stone,
            Nine for Mortal Men doomed to die,
              One for the Dark Lord on his dark throne
           In the Land of Mordor where the Shadows lie.
               One Ring to rule them all, One Ring to find them,
               One Ring to bring them all and in the darkness bind them
           In the Land of Mordor where the Shadows lie.
           
FOREWORD

This tale grew


In [ ]:
# Vectorize the text by converting the string into a numerical form.
sample_text = ['abcdefghij', 'wxyz']
num_chars = tf.strings.unicode_split(sample_text, input_encoding="UTF-8")
num_chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g', b'h', b'i', b'j'],
 [b'w', b'x', b'y', b'z']]>

In [ ]:
id_from_characters = tf.keras.layers.StringLookup(vocabulary=list(characters))
ids = id_from_characters(num_chars)
ids

<tf.RaggedTensor [[54, 55, 56, 57, 58, 59, 60, 61, 62, 63], [76, 77, 78, 79]]>

In [ ]:
characters_from_id = tf.keras.layers.StringLookup(vocabulary=id_from_characters.get_vocabulary(), invert=True)
characters = characters_from_id(ids)
characters

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g', b'h', b'i', b'j'],
 [b'w', b'x', b'y', b'z']]>

In [ ]:
# Convert the corresponding string ids back into text form.
def text_from_id(ids):
    return tf.strings.reduce_join(characters_from_id(ids), axis=-1)

In [ ]:
all_text_id = id_from_characters(tf.strings.unicode_split(corpus_raw, 'UTF-8'))
all_text_id

<tf.Tensor: shape=(1021058,), dtype=int64, numpy=array([45, 61, 71, ..., 32, 10,  1])>

In [ ]:
# Try to print out the first 25 characters from raw text via tensor slices.
id_dataset = tf.data.Dataset.from_tensor_slices(all_text_id)
for curr_id in id_dataset.take(25):
    print(characters_from_id(curr_id).numpy().decode('utf-8'))

T
h
r
e
e
 
R
i
n
g
s
 
f
o
r
 
t
h
e
 
E
l
v
e
n


In [ ]:
# Now build a sequence of the first 200 characters.
sequence_len = 200
each_example = len(corpus_raw)
sequences = id_dataset.batch(sequence_len+1, drop_remainder=True)

for curr_seq in sequences.take(1):
    print(characters_from_id(curr_seq))

tf.Tensor(
[b'T' b'h' b'r' b'e' b'e' b' ' b'R' b'i' b'n' b'g' b's' b' ' b'f' b'o'
 b'r' b' ' b't' b'h' b'e' b' ' b'E' b'l' b'v' b'e' b'n' b'-' b'k' b'i'
 b'n' b'g' b's' b' ' b'u' b'n' b'd' b'e' b'r' b' ' b't' b'h' b'e' b' '
 b's' b'k' b'y' b',' b'\n' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b' ' b'S' b'e' b'v' b'e' b'n' b' ' b'f' b'o'
 b'r' b' ' b't' b'h' b'e' b' ' b'D' b'w' b'a' b'r' b'f' b'-' b'l' b'o'
 b'r' b'd' b's' b' ' b'i' b'n' b' ' b't' b'h' b'e' b'i' b'r' b' ' b'h'
 b'a' b'l' b'l' b's' b' ' b'o' b'f' b' ' b's' b't' b'o' b'n' b'e' b','
 b'\n' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b'N'
 b'i' b'n' b'e' b' ' b'f' b'o' b'r' b' ' b'M' b'o' b'r' b't' b'a' b'l'
 b' ' b'M' b'e' b'n' b' ' b'd' b'o' b'o' b'm' b'e' b'd' b' ' b't' b'o'
 b' ' b'd' b'i' b'e' b',' b'\n' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b' ' b'O' b'n' b'e' b' ' b'f' b'o' b'r' b' '
 b't' b'h' b'e' b' ' b'D' b'a' b'r' b'k' b' ' b'L' b'o' b'r' b'

In [ ]:
for curr_seq in sequences.take(5):
    print(text_from_id(curr_seq).numpy())

b'Three Rings for the Elven-kings under the sky,\n               Seven for the Dwarf-lords in their halls of stone,\n            Nine for Mortal Men doomed to die,\n              One for the Dark Lord on hi'
b's dark throne\n           In the Land of Mordor where the Shadows lie.\n               One Ring to rule them all, One Ring to find them,\n               One Ring to bring them all and in the darkness bind'
b' them\n           In the Land of Mordor where the Shadows lie.\n           \nFOREWORD\n\nThis tale grew in the telling, until it became a history of the Great War of the Ring and included many glimpses of t'
b'he yet more ancient history that preceded it. It was begun soon after _The Hobbit_ was written and before its publication in 1937; but I did not go on with this sequel, for I wished first to complete a'
b'nd set in order the mythology and legends of the Elder Days, which had then been taking shape for some years. I desired to do this for my own satisfaction, and I had l

In [ ]:
def split_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
text_dataset = sequences.map(split_target)
for input_text, target_text in text_dataset.take(1):
    print("Input: ", text_from_id(input_text).numpy())
    print("Target: ", text_from_id(target_text).numpy())

Input:  b'Three Rings for the Elven-kings under the sky,\n               Seven for the Dwarf-lords in their halls of stone,\n            Nine for Mortal Men doomed to die,\n              One for the Dark Lord on h'
Target:  b'hree Rings for the Elven-kings under the sky,\n               Seven for the Dwarf-lords in their halls of stone,\n            Nine for Mortal Men doomed to die,\n              One for the Dark Lord on hi'


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    text_dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 200), dtype=tf.int64, name=None), TensorSpec(shape=(64, 200), dtype=tf.int64, name=None))>

In [ ]:
vocab = sorted(set(corpus_raw))
vocab_size = len(vocab)
embedding_dimension=256
rnn_units=1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(id_from_characters.get_vocabulary()),
    embedding_dim=embedding_dimension,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 200, 91) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  23296     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  93275     
                                                                 
Total params: 4,054,875
Trainable params: 4,054,875
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([89,  8, 20,  0, 88, 26, 82, 88, 41, 50, 66,  2, 79, 16, 58,  2, 18,
       83, 25, 58, 26, 45, 12, 60, 65,  0, 47, 71, 52, 71, 29, 34, 55, 76,
       39, 80, 47, 76, 89,  6, 65, 38, 36, 35, 58, 65, 16, 77, 62, 80, 74,
       76, 86, 56,  4,  1, 63, 85, 66, 56, 18, 52, 10, 84, 67, 16, 40, 57,
       37, 41, 17, 24, 63, 20, 43,  5, 44,  2, 63, 23,  4,  7, 60, 84, 47,
       36, 55, 80, 60, 16, 85, 70, 45, 74, 56, 75, 76, 22, 85, 86, 36, 33,
       72, 33, 52, 53, 36, 11, 30,  6, 83, 34, 63, 37,  8, 48, 28,  1, 72,
       77, 58, 16, 60, 75, 64, 44, 64, 77,  2, 30, 70, 40, 89, 13, 46, 43,
       39, 41, 61, 37, 25, 56,  5, 25, 31, 90, 89, 29, 77, 69, 30, 13, 53,
       62, 90,  9, 49, 77, 14, 82, 30, 58, 75, 45, 27, 26, 56, 32, 16, 66,
       84,  9, 35, 77,  6, 46, 35, 42, 30, 41, 62, 32, 31, 28, 54, 15, 23,
       28, 10, 54, 65, 12, 82, 16, 38, 69, 14, 44, 52,  3])

In [ ]:
print("Input:\n", text_from_id(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_id(sampled_indices).numpy())

Input:
 b' the background of their minds and memories that was very similar. They understood one another remarkably well, very much better than a hobbit would understand, say, a Dwarf, or an Orc, or even an Elf'

Next Char Predictions:
 b'\xc3\xbb,8[UNK]\xc3\xbaA\xc3\xa2\xc3\xbaPYm z4e 6\xc3\xa4?eAT0gl[UNK]Vr_rDIbwN\xc3\x93Vw\xc3\xbb(lMKJel4xi\xc3\x93uw\xc3\xadc"\nj\xc3\xabmc6_.\xc3\xa9n4OdLP5=j8R\'S j;")g\xc3\xa9VKb\xc3\x93g4\xc3\xabqTucvw:\xc3\xab\xc3\xadKHsH_`K/E(\xc3\xa4IjL,WC\nsxe4gvkSkx EqO\xc3\xbb1URNPhL?c\'?F\xe2\x80\x93\xc3\xbbDxpE1`i\xe2\x80\x93-Xx2\xc3\xa2EevTBAcG4m\xc3\xa9-Jx(UJQEPiGFCa3;C.al0\xc3\xa24Mp2S_!'


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)
tf.exp(example_batch_mean_loss).numpy()

Prediction shape:  (64, 200, 91)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.5118217, shape=(), dtype=float32)


91.08761

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(dataset, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
79/79 [==============================] - 25s 263ms/step - loss: 2.9964
Epoch 2/100
79/79 [==============================] - 22s 261ms/step - loss: 2.2272
Epoch 3/100
79/79 [==============================] - 23s 266ms/step - loss: 1.9535
Epoch 4/100
79/79 [==============================] - 22s 265ms/step - loss: 1.7351
Epoch 5/100
79/79 [==============================] - 22s 266ms/step - loss: 1.5642
Epoch 6/100
79/79 [==============================] - 23s 266ms/step - loss: 1.4430
Epoch 7/100
79/79 [==============================] - 23s 268ms/step - loss: 1.3584
Epoch 8/100
79/79 [==============================] - 22s 261ms/step - loss: 1.2947
Epoch 9/100
79/79 [==============================] - 22s 262ms/step - loss: 1.2441
Epoch 10/100
79/79 [==============================] - 22s 265ms/step - loss: 1.2029
Epoch 11/100
79/79 [==============================] - 22s 265ms/step - loss: 1.1663
Epoch 12/100
79/79 [==============================] - 22s 266ms/step - loss: 1.1339
E

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, characters_from_id, id_from_characters, 1.0)
one_step_model1 = OneStep(model, characters_from_id, id_from_characters, 0.9)
one_step_model2 = OneStep(model, characters_from_id, id_from_characters, 0.8)
one_step_model3 = OneStep(model, characters_from_id, id_from_characters, 0.7)
one_step_model4 = OneStep(model, characters_from_id, id_from_characters, 0.6)
one_step_model5 = OneStep(model, characters_from_id, id_from_characters, 0.5)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['There'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

There they have a way of beating. Sam gived my heart that Gandalf chose many minutes here and there the darkness the horsems of war, free down under the Mines, to us of the sky above the longer steep and the Forest and most stoods in his pocket.
     His watch was nearly as quickly as they could. Soon after half way account with light of almost silent, was now too imagined. The birds was lords. There was a fire or craveling through a clink and noon. Then setting it aside, he ran to the edge and peered over the menect of Balin and its fall; the passage twisted tree miles, tired but neither old home and bright. 'There are no hope that for this is the worthy meant, for it is eight,' said stood, and then took something in it. He flugged away from the edges of Amon Hen and uncamed his pocket. They started in their horses listening; and behind it rolling here and the Elves dear to be in his blanket. 'Very seath you are right in the looks and placest of things little a sigh. I have never seen

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['There'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model1.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

There they would ever a rest, remembered, but when these are elf-voice, and bent down the river and the flat suspicious of Buckland (alwer-pressed or heard the northern and bore many places. Would not help us to keep so stroken than five gone. Bilbo among the Wise, soon after I am, or we saw against the hunter.'
     Aragorn sprang away from the hill, they faded, and suddenly she passed through Arche-toithern-sky. There was no sound. Trembling he looked suddenly from a fire. On the near he began his feet. He felt that he had not heard no sign of any travels and boltestors in Elves; end on the landing stone had fallen in water-doings of night, and their forgiss and dwarves eastward from their thoughts. Frodo stood in the distance they heard again the quirt about them. They could see nothing that minal contained many cliffs, who asked the stone or juddle. Yet should be miles the first rainon and be glad indeed; and now he wields resist him. If gave way off three or of us to take Sam. 'I 

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['There'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model2.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

There they have a talk agreen. In the dept since he seemed to remember something; and in the end she departed with young hobbit and reeds by the slope, was now the lord of many sturdy men, and their forgs inco ofening was pass in the wind. An eastward peneats statuen many plains, and from its pages came dimly in the sun.
     The sun had gone far and wide that Glóin embarken of a ross, sand somewhere, on the edge of the river-valley and set up before them. The river suddenly thin and seek a face was strength.
     `_Care we first entered Merry's! ' said Pippin. 'You won't have a naped your boots of my house,' said Boromir. `It is a high present from all the arswer; and now then a northern lands was left behind, even in Rivendell. We have called the Ring down to death.'
     'Destroye_?' cried Frodo. `And what had become of the Mines? Hear will shade a friend of mine friends. Turned Saruman awáre. I could not ride among the streams. But your ring is in his book,' said Aragorn, 'foind in

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['There'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model3.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

There they had all passed through, it was already his by right. This was the song as the last house on the others; though he kept out through a deep ravine that led them somewhere to large. He would soon now, but only remember it argulatching the Ford by the road; so on an edge must need be some great mountains.
     Gandalf paused astonished and looked at the stone, and listening for the rustle of valours, and her sight moon; and the flies began to find the Grey Havens, from over the mountains and the door. But when he looked about up and rather shut, and it grows near the shore below.
     'They'll be cleared quietly. I never heard of anything yet. In some I naid was dead. And you must go and saw it as my letter. I didn't like it aside, I feel like this, maybe, we were all sake you.'
     'I know,' said Sam. 'It's a pity Mr. Baggins drows ahead difficult to the legend, and it would be a dark fire willow!'
     'I think I can produce him.'
     'Where is he?' says some ordinary hobbit

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['There'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model4.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

There they have a pain of a bone of sight alone, all those that he saw was shadowed and falling round again in the mountains. Troll, water for Bilbo, there was a ribbont-bood, and it had been built in the dusk of Amroth, where it is never a rule. But what was it that the Mirror slew much quiet! ' said Sam. `I will diminish, and go ay such a thing could stand to be pares! I wish you were not going into the I new hours. You have seen one of the lore-masters,' said Gandalf. `The passage is blocked behind us now and there the River seems to the point. No folk could they see, and they will not take the Ring itself to wait until nightfall. After all his father was a Baggins. A decent respectally the Sickle was or joint out again. He said that Frodo was looking across lands he had never seen careful that they did not mean to go on again full of plain grey hill, and strode on the bank at the sides, and looked up and down a large accourt:

     _For A LOGAffain! ' He murmed out of a decring. I 

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['There'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model5.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

There they have a passion for mushrooms, surpassing even him known to me. I have known your friend is!' said Frodo. 'I am not a Baggins, the Nine Sea, and not when he has a pit for certain. It was round solemold, if you do. Before the troll-folk is useless. And if I'me never heard of anything like this rider as stop misings. I wish to learn what has it got in its pocketses?" he says as soon as it should be, and always dark, and cast them on a fall away sharp ears. Sweeting!

     Nothing could be seen all round the house but falling water was loud, and the evening was filled with a thick hedge on the lane. The perish looked straight at the sky was broken over them. The sun was white hummer and looked against the draught: there were still meaning on the shoulder with the hurrying river. In the midst of it had been given the thing hope spoken of.'
     `And if that is what you well to do with it?' I'll talk.'
     'And no wonder that don't pass I have learned much,' he said in a low voic

#Text generation was also attempted with [Max Woolf's textgenrnn](https://github.com/minimaxir/textgenrnn), using a smaller text dataset from the script of the Bee Movie. All RNN settings used were the defaults from [the Colaboratory Notebook](https://drive.google.com/file/d/1mMKGnVxirJnqDViH7BDJxFqWrsXlPSoK/view), with the exception of the following changes:
#In the training configuration (train_cfg), the number of epochs was increased from 30 to 50, the proportion of input data to train (train_size) was increased from 0.8 to 0.9, and the dropout was increased from 0.0 to 0.1 (to counterbalance the increase in proportion of input data used for training).

# See the raw outputs document for the text generated from training textgenrnn on the Bee Movie script.

